In [16]:
import pandas as pd
from sqlalchemy import create_engine
import qgrid
import numpy as np

### Extract CSVs into DataFrames

In [3]:
premise_file = "beers_brewery_locations_city_state.csv"
premise = pd.read_csv(premise_file)
del premise['Unnamed: 0']
premise.head()

,Beer,Total_Votes,Rating,Brewery,Beer_Style,ABV,Region,Latitude,Longitude,City,State
0,Bitches' Bank,234,4.41,18th Street Brewery - Gary Taproom,Russian Imperial Stout,12.00%,great-lakes,41.599057,-87.267179,Gary,IN
1,Marshmallow Handjee,"1,682",4.73,3 Floyds Brewing Co.,Russian Imperial Stout,15.00%,great-lakes,41.535525,-87.516811,Munster,IN
2,Pear Bear,298,4.36,3 Floyds Brewing Co.,American Wild Ale,7.50%,great-lakes,41.535525,-87.516811,Munster,IN
3,Barrel Aged Behemoth,474,4.36,3 Floyds Brewing Co.,American Barleywine,12.50%,great-lakes,41.535525,-87.516811,Munster,IN
4,French Vanilla Militia,35,4.51,3 Floyds Brewing Co.,Russian Imperial Stout,15.00%,great-lakes,41.535525,-87.516811,Munster,IN


In [8]:
premise.dtypes

Beer            object
Total_Votes     object
Rating         float64
Brewery         object
Beer_Style      object
ABV             object
Region          object
Latitude       float64
Longitude      float64
City            object
State           object
dtype: object

In [22]:
premise = premise.replace({'ABV': {'ld Ale': np.nan, 'eywine': np.nan, 'd Beer': np.nan, 'Stout': np.nan, 
                        'Saison': np.nan, 'Gueuze': np.nan, 'al IPA': np.nan}})

In [23]:
qgrid.show_grid(premise)

QgridWidget(grid_options={'fullWidthRows': True, 'syncColumnCellResize': True, 'forceFitColumns': True, 'defau…

In [10]:
premise['Total_Votes'] = premise['Total_Votes'].str.replace(",","").astype(int)

In [24]:
premise['ABV'] = premise['ABV'].str.replace("%","").astype(float)

In [25]:
premise.dtypes

Beer            object
Total_Votes      int64
Rating         float64
Brewery         object
Beer_Style      object
ABV            float64
Region          object
Latitude       float64
Longitude      float64
City            object
State           object
dtype: object

In [29]:
us_states = pd.read_csv('states.csv')

In [30]:
us_states.head()

,State,Abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


### Create database connection

In [26]:
#engine = create_engine("mysql://root@localhost/customer_db")
engine = create_engine("sqlite:///beer_brewery_db.sqlite")
conn = engine.connect()

In [27]:
#rds_connection_string = "<inser user name>:<insert password>@127.0.0.1/customer_db"
#engine = create_engine(f'mysql://{rds_connection_string}')

In [28]:
# Confirm tables
engine.table_names()

['beer']

### Load DataFrames into database

In [6]:
premise.head()

,Beer,Total_Votes,Rating,Brewery,Beer_Style,ABV,Region,Latitude,Longitude,City,State
0,Bitches' Bank,234,4.41,18th Street Brewery - Gary Taproom,Russian Imperial Stout,12.00%,great-lakes,41.599057,-87.267179,Gary,IN
1,Marshmallow Handjee,"1,682",4.73,3 Floyds Brewing Co.,Russian Imperial Stout,15.00%,great-lakes,41.535525,-87.516811,Munster,IN
2,Pear Bear,298,4.36,3 Floyds Brewing Co.,American Wild Ale,7.50%,great-lakes,41.535525,-87.516811,Munster,IN
3,Barrel Aged Behemoth,474,4.36,3 Floyds Brewing Co.,American Barleywine,12.50%,great-lakes,41.535525,-87.516811,Munster,IN
4,French Vanilla Militia,35,4.51,3 Floyds Brewing Co.,Russian Imperial Stout,15.00%,great-lakes,41.535525,-87.516811,Munster,IN


In [7]:
#premise.to_sql(name='premis', con=engine, if_exists='append', index=False)

In [8]:
#pd.read_sql('belly_button_biodiversity.sqlite')

In [31]:
premise = premise.reset_index(drop=False)

#premise = premise.rename(index=str, columns={"index": "beer_key"})

premise.head()

,index,Beer,Total_Votes,Rating,Brewery,Beer_Style,ABV,Region,Latitude,Longitude,City,State
0,0,Bitches' Bank,234,4.41,18th Street Brewery - Gary Taproom,Russian Imperial Stout,12.0,great-lakes,41.599057,-87.267179,Gary,IN
1,1,Marshmallow Handjee,1682,4.73,3 Floyds Brewing Co.,Russian Imperial Stout,15.0,great-lakes,41.535525,-87.516811,Munster,IN
2,2,Pear Bear,298,4.36,3 Floyds Brewing Co.,American Wild Ale,7.5,great-lakes,41.535525,-87.516811,Munster,IN
3,3,Barrel Aged Behemoth,474,4.36,3 Floyds Brewing Co.,American Barleywine,12.5,great-lakes,41.535525,-87.516811,Munster,IN
4,4,French Vanilla Militia,35,4.51,3 Floyds Brewing Co.,Russian Imperial Stout,15.0,great-lakes,41.535525,-87.516811,Munster,IN


In [32]:
premise.to_sql(name='beer', con=engine, if_exists='replace', index=False)

In [34]:
us_states.to_sql(name='us_states', con=engine, if_exists='replace', index=False)

In [33]:
pd.read_sql_query('select * from beer', con=engine)

,index,Beer,Total_Votes,Rating,Brewery,Beer_Style,ABV,Region,Latitude,Longitude,City,State
0,0,Bitches' Bank,234,4.41,18th Street Brewery - Gary Taproom,Russian Imperial Stout,12.0,great-lakes,41.599057,-87.267179,Gary,IN
1,1,Marshmallow Handjee,1682,4.73,3 Floyds Brewing Co.,Russian Imperial Stout,15.0,great-lakes,41.535525,-87.516811,Munster,IN
2,2,Pear Bear,298,4.36,3 Floyds Brewing Co.,American Wild Ale,7.5,great-lakes,41.535525,-87.516811,Munster,IN
3,3,Barrel Aged Behemoth,474,4.36,3 Floyds Brewing Co.,American Barleywine,12.5,great-lakes,41.535525,-87.516811,Munster,IN
4,4,French Vanilla Militia,35,4.51,3 Floyds Brewing Co.,Russian Imperial Stout,15.0,great-lakes,41.535525,-87.516811,Munster,IN
5,5,Dreadnaught IPA,6128,4.40,3 Floyds Brewing Co.,American Imperial IPA,9.5,great-lakes,41.535525,-87.516811,Munster,IN
6,6,Permanent Funeral,3013,4.50,3 Floyds Brewing Co.,American Imperial IPA,10.5,great-lakes,41.535525,-87.516811,Munster,IN
7,7,BBADL (Bourbon Barrel Aged Dark Lord Imperial ...,932,4.56,3 Floyds Brewing Co.,Russian Imperial Stout,15.0,great-lakes,41.535525,-87.516811,Munster,IN
8,8,3 Floyds / The Bruery - Rue D'Floyd,315,4.43,3 Floyds Brewing Co.,American Imperial Porter,14.4,great-lakes,41.535525,-87.516811,Munster,IN
9,9,Chemtrailmix,117,4.71,3 Floyds Brewing Co.,Russian Imperial Stout,15.0,great-lakes,41.535525,-87.516811,Munster,IN


In [35]:
pd.read_sql_query('select * from us_states', con=engine).head()

,State,Abbreviation
0,Alabama,AL
1,Alaska,AK
2,Arizona,AZ
3,Arkansas,AR
4,California,CA


In [37]:
conn.close()